In [364]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf

In [365]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

sample = pd.read_csv("SampleSubmission.csv")

In [366]:
#Combine test and train into one file
train['source']='train'
test['source']='test'
data = pd.concat([train, test],ignore_index=True)
print (train.shape, test.shape, data.shape)

(4990, 14) (3532, 13) (8522, 14)


In [367]:
data['Product_Weight'] = data['Product_Weight'].fillna(0)
sum(data['Product_Weight'].isnull())

0

In [368]:
data['Supermarket_Size'] = data['Supermarket _Size'].fillna('NaN')
sum(data['Supermarket_Size'].isnull())

0

In [369]:
data['Supermarket_Opening_Year'] = 2016 - data['Supermarket_Opening_Year']

In [370]:
data.head()

,Product_Fat_Content,Product_Identifier,Product_Price,Product_Shelf_Visibility,Product_Supermarket_Identifier,Product_Supermarket_Sales,Product_Type,Product_Weight,Supermarket _Size,Supermarket_Identifier,Supermarket_Location_Type,Supermarket_Opening_Year,Supermarket_Type,source,Supermarket_Size
0,Low Fat,DRA12,357.54,0.068535,DRA12_CHUKWUDI010,709.08,Soft Drinks,11.6,NaN,CHUKWUDI010,Cluster 3,11,Grocery Store,train,NaN
1,Low Fat,DRA12,355.79,0.040912,DRA12_CHUKWUDI013,6381.69,Soft Drinks,11.6,High,CHUKWUDI013,Cluster 3,22,Supermarket Type1,train,High
2,Low Fat,DRA12,350.79,0.041178,DRA12_CHUKWUDI017,6381.69,Soft Drinks,11.6,NaN,CHUKWUDI017,Cluster 2,2,Supermarket Type1,train,NaN
3,Low Fat,DRA12,355.04,0.041113,DRA12_CHUKWUDI018,2127.23,Soft Drinks,11.6,Medium,CHUKWUDI018,Cluster 3,0,Supermarket Type2,train,Medium
4,Ultra Low fat,DRA12,354.79,0.000000,DRA12_CHUKWUDI035,2481.77,Soft Drinks,11.6,Small,CHUKWUDI035,Cluster 2,5,Supermarket Type1,train,Small


In [371]:
cols = ['Product_Price','Product_Shelf_Visibility','Product_Weight','Supermarket_Opening_Year']
data[cols] = data[cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [372]:
data['Product_Identifier'].nunique()

1559

In [373]:
#data['Supermarket_Opening_Year'].hist(bins=20)
#age_buckets = tf.feature_column.bucketized_column(age, boundaries=[0,5,10,15,20,25,30])

In [374]:
#Drop the columns which have been converted to different types:
data.drop(['Product_Supermarket_Identifier','Supermarket _Size'],axis=1,inplace=True)

In [375]:
#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

In [376]:
test.drop(['Product_Supermarket_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

C:\Users\mo.yosiwealth\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\mo.yosiwealth\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [377]:
y = train['Product_Supermarket_Sales']
X = train.drop(['Product_Supermarket_Sales'],axis=1)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [378]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3493 entries, 3100 to 2732
Data columns (total 11 columns):
Product_Fat_Content          3493 non-null object
Product_Identifier           3493 non-null object
Product_Price                3493 non-null float64
Product_Shelf_Visibility     3493 non-null float64
Product_Type                 3493 non-null object
Product_Weight               3493 non-null float64
Supermarket_Identifier       3493 non-null object
Supermarket_Location_Type    3493 non-null object
Supermarket_Opening_Year     3493 non-null float64
Supermarket_Type             3493 non-null object
Supermarket_Size             3493 non-null object
dtypes: float64(4), object(7)
memory usage: 327.5+ KB


In [379]:
# continuous feature_columns for the continuous values using numeric_column

price = tf.feature_column.numeric_column('Product_Price')
visibility = tf.feature_column.numeric_column('Product_Shelf_Visibility')
weight = tf.feature_column.numeric_column('Product_Weight')
age = tf.feature_column.numeric_column('Supermarket_Opening_Year')

In [380]:
# tf.feature_columns for the categorical values. Using vocabulary lists or just use hash buckets.

assigned_group_FC = tf.feature_column.categorical_column_with_hash_bucket('Product_Fat_Content', hash_bucket_size=10)
assigned_group_PI = tf.feature_column.categorical_column_with_hash_bucket('Product_Identifier', hash_bucket_size=10)
assigned_group_PT = tf.feature_column.categorical_column_with_hash_bucket('Product_Type', hash_bucket_size=10)
assigned_group_SS = tf.feature_column.categorical_column_with_hash_bucket('Supermarket_Size', hash_bucket_size=10)
assigned_group_SI = tf.feature_column.categorical_column_with_hash_bucket('Supermarket_Identifier', hash_bucket_size=10)
assigned_group_LT = tf.feature_column.categorical_column_with_hash_bucket('Supermarket_Location_Type', hash_bucket_size=10)
assigned_group_ST = tf.feature_column.categorical_column_with_hash_bucket('Supermarket_Type', hash_bucket_size=10)

In [381]:
embedded_group_FC = tf.feature_column.embedding_column(assigned_group_FC, dimension=3)
embedded_group_PI = tf.feature_column.embedding_column(assigned_group_PI, dimension=16)
embedded_group_PT = tf.feature_column.embedding_column(assigned_group_PT, dimension=16)
embedded_group_SS = tf.feature_column.embedding_column(assigned_group_SS, dimension=3)
embedded_group_SI = tf.feature_column.embedding_column(assigned_group_SI, dimension=10)
embedded_group_LT = tf.feature_column.embedding_column(assigned_group_LT, dimension=3)
embedded_group_ST = tf.feature_column.embedding_column(assigned_group_ST, dimension=4)

In [388]:
feat_cols = [price, visibility, weight, age,embedded_group_SS, #embedded_group_SI, , embedded_group_FC,
              embedded_group_ST] #, embedded_group_PIembedded_group_LT, embedded_group_PT, 

## Create Input Function

In [389]:
# Batch_size is up to you. But do make sure to shuffle!

input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [390]:
# Create model with tf.estimator
# Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create 
# embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)


model = tf.estimator.DNNRegressor(hidden_units=[14,12,10,8,10,12,14],feature_columns=feat_cols)

#model = tf.estimator.LinearRegressor(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\MOB005~1.YOS\\AppData\\Local\\Temp\\tmpihrykxhp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000024509A6DE80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [391]:
# Train your model on the data, for at least 5000 steps.

model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\MOB005~1.YOS\AppData\Local\Temp\tmpihrykxhp\model.ckpt.
INFO:tensorflow:loss = 538558400.0, step = 1
INFO:tensorflow:global_step/sec: 94.9383
INFO:tensorflow:loss = 107471650.0, step = 101 (1.057 sec)
INFO:tensorflow:global_step/sec: 144.605
INFO:tensorflow:loss = 87772824.0, step = 201 (0.697 sec)
INFO:tensorflow:global_step/sec: 143.049
INFO:tensorflow:loss = 100209970.0, step = 301 (0.701 sec)
INFO:tensorflow:global_step/sec: 150.823
INFO:tensorflow:loss = 115824920.0, step = 401 (0.664 sec)
INFO:tensorflow:global_step/sec: 137.968
INFO:tensorflow:loss = 177342820.0, step = 501 (0.726 sec)
INFO:tensorflow:global_step/sec: 144.414
INFO:tensorflow:loss = 228590800.0, step = 601 (0.695 sec)
INFO:ten

In [392]:
# Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False.

predict_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)


# Use model.predict() and pass in your input function. This will produce a generator of predictions, 
# which you can then transform into a list, with list()

pred_gen = model.predict(predict_input_func)

predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\MOB005~1.YOS\AppData\Local\Temp\tmpihrykxhp\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [393]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

from sklearn.metrics import mean_squared_error

mean_squared_error(y_test,final_preds)**0.5

3032.929315890657

In [348]:
predict_new_data_func = tf.estimator.inputs.pandas_input_fn(x=test,batch_size=10,num_epochs=1,shuffle=False)

pred = model.predict(predict_new_data_func)

new_data_predictions = list(pred)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\MOB005~1.YOS\AppData\Local\Temp\tmpu3q9qnqr\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [349]:
new_preds = []
for pred in new_data_predictions:
    new_preds.append(pred['predictions'])

In [350]:
submission = pd.DataFrame({'Product_Supermarket_Identifier':sample['Product_Supermarket_Identifier'],
                                  'Product_Supermarket_Sales': new_preds},
                                 columns=['Product_Supermarket_Identifier','Product_Supermarket_Sales'])

#to csv
submission.to_csv("I'm tired.csv",index=False)